### Imports necesarios

In [1]:
# import keras.losses
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import cv2
import config
import pickle
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, losses, optimizers, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

### Cargar datos de entrenamiento

In [2]:
def load_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)


def load_training_data():
    df_training_data = load_pickle(config.TRAINING_FILE)

    imgs = df_training_data["image_name"].values

    training_imgs = []
    training_labels = []

    print(f"## Loading {len(imgs)} training images: ", flush=True)
    for img_file in tqdm(imgs, total=len(imgs)):
        im = cv2.imread(f"{config.TRAINING_IMGS}{img_file}", cv2.IMREAD_GRAYSCALE)
        is_male = df_training_data.loc[df_training_data["image_name"] == img_file]["Male"].values[0] == "male"
        training_imgs.append(im)
        training_labels.append([1, 0] if is_male else [0, 1])

    return np.asarray(training_imgs) / 255, np.asarray(training_labels)

# im = plt.imread(f"{config.TRAINING_IMGS}20803.jpg")
# img_width, img_height = im.shape
# pixel_count = img_width * img_height
#
# img_names = df['image_name'].values.reshape(1, df['image_name'].values.size)[0]
# X_train = np.array([plt.imread(f"{config.TRAINING_IMGS}{img_name}") for img_name in img_names]) / 255
# Y_train = np.where(df["Male"] == "male", 1, 0).T

def load_validation_data():
    df_validation_labels = load_pickle(config.VALIDATION_LABELS)
    df_validation_bboxes = load_pickle(config.VALIDATION_BBOX)

    imgs = sorted(glob(f"{config.VALIDATION_IMGS}*.jpg"))

    val_imgs = []
    val_labels = []


    print(f"## Loading {len(imgs)} validation images: ", flush=True)
    for img_file in tqdm(imgs, total=len(imgs)):

        image_name = os.path.basename(img_file)

        big_image = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2GRAY)

        bbox = df_validation_bboxes.loc[df_validation_bboxes["image_id"] == image_name]
        cut_image = big_image[bbox['y_top'].values[0] : bbox['y_top'].values[0] + bbox['height'].values[0],
                    bbox['x_left'].values[0] : bbox['x_left'].values[0] + bbox['width'].values[0]]
        try:
            cut_image = cv2.resize(cut_image, [256, 256], cv2.INTER_AREA)
            val_imgs.append(cut_image)
            is_male = df_validation_labels.loc[df_validation_labels["image_name"] == image_name]["Male"].values[0] == "male"
            val_labels.append([1, 0] if is_male else [0, 1])
        except cv2.error:
            print(f"Failed to resize image: {image_name}")

    return np.asarray(val_imgs) / 255, np.asarray(val_labels)


def define_model(img_width, img_height):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation=tf.nn.relu,  input_shape=(img_width, img_height, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation=tf.nn.relu))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation=tf.nn.relu))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(.5))
    model.add(Dense(128, activation=tf.nn.relu))
    model.add(Dense(32, activation=tf.nn.relu))
    model.add(Dense(2, activation=tf.nn.softmax))

    loss = "categorical_crossentropy"
    optimizer = optimizers.SGD(learning_rate=.01)
    metrics = ["accuracy"]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, Y_train, X_val, Y_val, batch_size, epochs):
    return model.fit(X_train, Y_train, verbose=1, validation_data=(X_val, Y_val), batch_size=batch_size, epochs=epochs)


In [3]:
X_train, Y_train = load_training_data()
X_val, Y_val = load_validation_data()
width, height = X_train[0].shape
model = define_model(width, height)

## Loading 9914 training images: 


100%|██████████| 9914/9914 [00:11<00:00, 880.00it/s]


## Loading 175 validation images: 


 28%|██▊       | 49/175 [00:00<00:00, 481.57it/s]

Failed to resize image: 035773.jpg


100%|██████████| 175/175 [00:00<00:00, 482.70it/s]


In [4]:
history = train_model(model, X_train, Y_train, X_val, Y_val, 64, 10)


Epoch 1/10
155/155 [==============================] - 19s 101ms/step - loss: 0.5286 - accuracy: 0.7690 - val_loss: 5.7364 - val_accuracy: 0.4770

In [5]:
model.save(config.MODEL_SAVE_FILE)


INFO:tensorflow:Assets written to: saved_model/gender-classification-model\assets


INFO:tensorflow:Assets written to: saved_model/gender-classification-model\assets
